In [1]:
# model class 

import torch 
from torch import nn 
from tqdm.auto import tqdm 

import json
from importlib import reload

import torch
import numpy as np
import pandas as pd 
from tqdm.auto import tqdm
from sklearn.metrics import balanced_accuracy_score, f1_score, confusion_matrix

from nn_module import BiLSTM_CRF

In [2]:
def data_label_split(data, label, train_size=0.8):
    randidx = np.arange(len(data))
    data_train, data_test = train_test_split(data, randidx, train_size)
    label_train, label_test = train_test_split(label, randidx, train_size)

    return data_train, data_test, label_train, label_test

def train_test_split(data, randidx, train_size):
    N = len(data)
    return [data[i] for i in randidx[:int(train_size*N)]], [data[i] for i in randidx[int(train_size*N):]]

def shuffle_data_label_lists(data, label):
    randidx = np.arange(len(data))
    np.random.shuffle(randidx)
    return [data[i] for i in randidx], [label[i] for i in randidx]

def embedding(data, token2emb, label2num):
    tokens_lst = []
    labels_lst = []

    for doc in tqdm(data):
        tokens = map(lambda x: x.replace('\t', '').replace('\n', '').replace(' ', ''), doc['tokens'])
        emb_tokens = torch.zeros(len(doc['tokens']), 300)
        for i, token in enumerate(tokens):
            if token in token2emb:
                emb_tokens[i] = torch.FloatTensor(token2emb[token])
            else:
                for it in token:
                    emb_tokens[i] += torch.FloatTensor(token2emb[it] if it in token2emb else [0]*300)

        tokens_lst.append(torch.FloatTensor(emb_tokens))
        labels_lst.append(torch.LongTensor([label2num[it] if it in label2num else 0 for it in doc['labels']]))
        
    return tokens_lst, labels_lst

def batch_split(X, Y, seq_len, batch_size=1000):
    x_batched = []
    y_batched = []
    seq_len_batched = []

    n = len(X)
    pointer = 0
    while pointer + batch_size < n:
        x_batched.append(X[pointer:pointer+batch_size])
        y_batched.append(Y[pointer:pointer+batch_size])
        seq_len_batched.append(seq_len[pointer:pointer+batch_size])
        pointer += batch_size 
    
    x_batched.append(X[pointer:])
    y_batched.append(Y[pointer:])
    seq_len_batched.append(seq_len[pointer:])

    return x_batched, y_batched, seq_len_batched

def fix_label_disbalance(data, labels):
    label_idxs = {}
    for i in range(len(data)):
        unique_labels = np.unique(labels[i])
        for lab in unique_labels:
            if lab in label_idxs:
                label_idxs[lab].append(i)
            else:
                label_idxs[lab] = [i]

    idxs = []
    count_o = len(label_idxs[0])
    del label_idxs[0]

    for it in label_idxs:
        scale = count_o // len(label_idxs[it])
        idxs += label_idxs[it]*scale

    np.random.shuffle(idxs)
    data = [data[i] for i in idxs]
    labels = [labels[i] for i in idxs]

    return data, labels

In [3]:
# create table for name and address
label2num = {
    'B-NAME_STUDENT': 1, 
    'I-NAME_STUDENT': 2, 
    'B-STREET_ADDRESS': 3, 
    'I-STREET_ADDRESS': 4, 
    'B-USERNAME': 5,
    'I-USERNAME': 6, 
    'B-ID_NUM': 7, 
    'I-ID_NUM': 8, 
    '<START>': 9, 
    '<STOP>': 10,
    'O': 0,
}

num2label = {
    0: 'O',
    1: 'B-NAME_STUDENT',  
    2: 'I-NAME_STUDENT',  
    3: 'B-STREET_ADDRESS',  
    4: 'I-STREET_ADDRESS',  
    5: 'B-USERNAME', 
    6: 'I-USERNAME',  
    7: 'B-ID_NUM',  
    8: 'I-ID_NUM', 
}

# encoding tokens and labels
with open('data/mixtral-8x7b-v1.json', 'r', encoding='utf-8') as f:
    data_1 = json.load(f)
with open('data/train.json', 'r', encoding='utf-8') as f:
    data_2 = json.load(f)
data = data_1 + data_2

reg_set = set(['B-URL_PERSONAL', 'I-URL_PERSONAL', 'B-EMAIL', 'I-EMAIL', 'B-PHONE_NUM', 'I-PHONE_NUM'])
for i in range(len(data)):
    data[i]['labels'] = ['O' if it in reg_set else it for it in data[i]['labels']]

np.random.seed(123)
data_idx = np.arange(len(data))
np.random.shuffle(data_idx)
data = [data[i] for i in data_idx]

train_size = 0.7
n = int(len(data)*train_size)
train_data, valid_data = data[:n], data[n:]

In [4]:
# load fastext
token2emb = {}
with open('wiki-news-300d-1M.vec', 'r', encoding='utf-8') as f:
    next(f)
    for it in tqdm(f):
        row = it.split(' ')
        token2emb[row[0]] = list(map(float, row[1:]))

# split data on train and valid
train_tokens, train_labels = embedding(train_data, token2emb, label2num)
valid_tokens, valid_labels = embedding(valid_data, token2emb, label2num)

# clearing token2emb memory
token2emb = None
import gc
gc.collect()

train_tokens, train_labels = fix_label_disbalance(train_tokens, train_labels)
valid_tokens, valid_labels = fix_label_disbalance(valid_tokens, valid_labels)

0it [00:00, ?it/s]

  0%|          | 0/6413 [00:00<?, ?it/s]

  0%|          | 0/2749 [00:00<?, ?it/s]

In [9]:
# init class weigths
class_weigths = torch.zeros(max(label2num.values())+1)
for i in range(len(class_weigths)):
    class_weigths[i] = sum([(it == i).sum().item() for it in train_labels])
class_weigths = 1 / class_weigths 
class_weigths /= class_weigths.sum()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

import nn_module
reload(nn_module)
from nn_module import BiLSTM_CRF

# fit lstm
model = BiLSTM_CRF(
    embedding_dim=300,
    hidden_size=32,
    nclasses=len(class_weigths),
    label2num=label2num,
    device=device,
)
# checkpoint = torch.load('saved_models/bi_lstm_long.pt')
# model.load_state_dict(checkpoint['model_state_dict'])
# model.optim.load_state_dict(checkpoint['optimizer_state_dict'])

model.fit(
    train_X=train_tokens,
    train_Y=train_labels,
    valid_X=valid_tokens,
    valid_Y=valid_labels,
    nepochs=20,
    lr=1e-3,
    device=device
)

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [7]:
# save model
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': model.optim.state_dict(),
    }, 'saved_models/bi_lstm.pt')

In [9]:
token2emb = {}
with open('wiki-news-300d-1M.vec', 'r', encoding='utf-8') as f:
    next(f)
    for it in tqdm(f):
        row = it.split(' ')
        token2emb[row[0]] = list(map(float, row[1:]))

valid_tokens, valid_labels = embedding(valid_data, token2emb, label2num)

# clearing token2emb memory
token2emb = None
import gc
gc.collect()

0it [00:00, ?it/s]

  0%|          | 0/2749 [00:00<?, ?it/s]

39

In [10]:
with torch.no_grad():
    valid_predict = []
    for batch_X in valid_tokens:
        predict = model(batch_X.to(device)).cpu()
        valid_predict.append(torch.argmax(predict, dim=1))
    valid_predict = torch.cat(valid_predict)
    valid_real = torch.cat(valid_labels)
    print(f'BA: {balanced_accuracy_score(valid_real, valid_predict):.3f}')

BA: 0.994


In [16]:
columns = [
    'O',
    'NAME_STUDENT',
    'STREET_ADDRESS',
    'USERNAME',
    'ID_NUM',
]

pd.DataFrame(confusion_matrix(valid_real, valid_predict), columns=columns)

,O,NAME_STUDENT,STREET_ADDRESS,USERNAME,ID_NUM
0,2096363,433,10,22,16
1,48,4734,2,3,0
2,0,0,6565,0,0
3,9,0,0,732,0
4,5,0,0,0,1036


In [8]:
with torch.no_grad():
    train_predict = []
    train_proba = []
    for batch_X in train_tokens:
        predict = model(batch_X.to(device)).cpu()
        train_proba.append(predict[:, 1])
        train_predict.append(torch.argmax(predict, dim=1))
    
    train_predict = torch.cat(train_predict)
    train_proba = torch.cat(train_proba)
    train_real = torch.cat(train_labels)

    print(f'BA: {balanced_accuracy_score(train_real, train_predict):.3f}')
    print(f'F1: {f1_score(train_real, train_predict, average="micro"):.3f}')

with torch.no_grad():
    valid_predict = []
    valid_proba = []
    for batch_X in valid_tokens:
        predict = model(batch_X.to(device)).cpu()
        valid_proba.append(predict[:, 1])
        valid_predict.append(torch.argmax(predict, dim=1))

    valid_predict = torch.cat(valid_predict)
    valid_proba = torch.cat(valid_proba)
    valid_real = torch.cat(valid_labels)

    print(f'BA: {balanced_accuracy_score(valid_real, valid_predict):.3f}')
    print(f'F1: {f1_score(valid_real, valid_predict, average="micro"):.3f}')

BA: 1.000
F1: 1.000


In [11]:
columns = [
    'O',
    'NAME_STUDENT',
    'STREET_ADDRESS',
    # 'USERNAME',
    # 'ID_NUM',
]

pd.DataFrame(confusion_matrix(valid_real, valid_predict), columns=columns)

,O,NAME_STUDENT,STREET_ADDRESS
0,1671929,130,12
1,103,3815,1
2,0,0,5364


In [17]:
token2emb = []
train_tokens = []
train_labels = []
test_tokens = []
test_labels = []

import gc
gc.collect()

0

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class_weigths = torch.FloatTensor([1,1,1,1,1])
model = BiLSTM(
    embedding_dim=300,
    hidden_size=32,
    nclasses=len(class_weigths),
    device=device,
    class_weigths=class_weigths
)

checkpoint = torch.load('saved_models/bi_lstm.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.optim.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()
model.to(device)
print()

In [5]:
token2emb = {}
with open('wiki-news-300d-1M.vec', 'r', encoding='utf-8') as f:
    next(f)
    for it in tqdm(f):
        row = it.split(' ')
        token2emb[row[0]] = list(map(float, row[1:]))

0it [00:00, ?it/s]

In [6]:
def get_predict_table(data, token2emb, num2label, label2num):
    def check_name(x):
        flag = False
        flag = x[0].isupper()
        for it in x[1:]:
            flag *= it.islower()
        return flag
    
    data_tokens, _ = embedding(data, token2emb, label2num)
    predict_table = [[], [], [], [], []] # document, token_idx, token, predict, real
    for i in tqdm(range(len(data))):
        with torch.no_grad():
            predict = torch.argmax(model.forward(data_tokens[i].to(device)), dim=1).cpu()
            predict = [num2label[it.item()] for it in predict]
            predict = ['O' if len(set(x)) < 2 else y for x, y in zip(data[i]['tokens'], predict)]
            predict_table[0] += [data[i]['document']]*len(predict)
            predict_table[1] += list(range(len(predict)))
            predict_table[2] += data[i]['tokens']
            predict_table[3] += predict
            predict_table[4] += data[i]['labels'] 
        
    predict_table = [[predict_table[0][i], predict_table[1][i], predict_table[2][i], predict_table[3][i], predict_table[4][i]] for i in range(len(predict_table[0]))]
    predict_table = pd.DataFrame(predict_table, columns=['document', 'token_i', 'token', 'predict', 'label'])
    predict_table = predict_table.loc[(predict_table.label != 'O') & (predict_table.token != "\n") & (predict_table.token != "\n\n") & (predict_table.token != "\t")].reset_index(drop=True)
    predict_table['prefix'] = 'B-'

    predict_table = predict_table.sort_values('document').reset_index(drop=True)

    # return predict_table.loc[predict_table.label.isin(set(['O', 'B-ID_NUM', 'I-ID_NUM', 'B-USERNAME', 'I-USERNAME']))].reset_index(drop=True)
    # return predict_table.loc[predict_table.label.isin(set(['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-STREET_ADDRESS', 'I-STREET_ADDRESS']))].reset_index(drop=True)
    return predict_table

In [8]:
# create table for name and address
label2num = {
    'B-NAME_STUDENT': 1, 
    'I-NAME_STUDENT': 1, 
    'B-STREET_ADDRESS': 2, 
    'I-STREET_ADDRESS': 2, 
    'B-USERNAME': 3,
    'I-USERNAME': 3, 
    'B-ID_NUM': 4, 
    'I-ID_NUM': 4, 
    
    'O': 0,
    'B-URL_PERSONAL': 0, 
    'I-URL_PERSONAL': 0, 
    'B-EMAIL': 0, 
    'I-EMAIL': 0,
    'B-PHONE_NUM': 0, 
    'I-PHONE_NUM': 0, 
}

num2label = {
    0: 'O',
    1: 'NAME_STUDENT', 
    2: 'STREET_ADDRESS', 
    3: 'USERNAME',
    4: 'ID_NUM',
}

import gc
with open('data/train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

predict_table = get_predict_table(data, token2emb, num2label, label2num)
gc.collect()

  0%|          | 0/6807 [00:00<?, ?it/s]

  0%|          | 0/6807 [00:00<?, ?it/s]

0

In [9]:
predict_table.head()

,document,token_i,token,predict,label,prefix
0,7,9,Nathalie,NAME_STUDENT,B-NAME_STUDENT,B-
1,7,10,Sylla,NAME_STUDENT,I-NAME_STUDENT,B-
2,7,482,Nathalie,NAME_STUDENT,B-NAME_STUDENT,B-
3,7,483,Sylla,NAME_STUDENT,I-NAME_STUDENT,B-
4,7,741,Nathalie,NAME_STUDENT,B-NAME_STUDENT,B-


In [10]:
label = predict_table['label'].str.slice(2).values 
predict = predict_table['predict'].values 
print(f'F1 score: {f1_score(label, predict, average="micro")}')

F1 score: 0.9199853854585314


In [11]:
for i in range(1, predict_table.shape[0]):
    if (predict_table.at[i-1, 'predict'] == predict_table.at[i, 'predict']) & (predict_table.at[i-1, 'document'] == predict_table.at[i, 'document']) & (predict_table.at[i, 'token_i'] - predict_table.at[i-1, 'token_i'] == 1):
        predict_table.at[i, 'prefix'] = 'I-'
predict_table['predict'] = predict_table['prefix'] + predict_table['predict']

print(f'F1 score: {f1_score(predict_table.label, predict_table.predict, average="micro")}')

F1 score: 0.7822433321154548


In [12]:
predict_table

,document,token_i,token,predict,label,prefix
0,7,9,Nathalie,B-NAME_STUDENT,B-NAME_STUDENT,B-
1,7,10,Sylla,I-NAME_STUDENT,I-NAME_STUDENT,I-
2,7,482,Nathalie,B-NAME_STUDENT,B-NAME_STUDENT,B-
3,7,483,Sylla,I-NAME_STUDENT,I-NAME_STUDENT,I-
4,7,741,Nathalie,B-NAME_STUDENT,B-NAME_STUDENT,B-
...,...,...,...,...,...,...
2732,15717,365,IV-8322,B-ID_NUM,B-ID_NUM,B-
2733,15717,964,IV-8322,B-ID_NUM,B-ID_NUM,B-
2734,19280,55,30407059,B-ID_NUM,I-ID_NUM,B-
2735,19280,54,Z.S.,B-ID_NUM,B-ID_NUM,B-


In [13]:
predict_table.loc[predict_table.predict != predict_table.label].groupby('predict', as_index=False).label.count()

,predict,label
0,B-ID_NUM,1
1,B-NAME_STUDENT,366
2,B-O,211
3,B-STREET_ADDRESS,10
4,B-USERNAME,1
5,I-O,7


In [12]:
predict_table

,document,token_i,token,predict,label,prefix,upper_start
0,7,9,Nathalie,B-NAME_STUDENT,B-NAME_STUDENT,B-,1
1,7,10,Sylla,I-NAME_STUDENT,I-NAME_STUDENT,I-,1
2,7,482,Nathalie,B-NAME_STUDENT,B-NAME_STUDENT,B-,1
3,7,483,Sylla,I-NAME_STUDENT,I-NAME_STUDENT,I-,1
4,7,741,Nathalie,B-NAME_STUDENT,B-NAME_STUDENT,B-,1
...,...,...,...,...,...,...,...
2639,22147,457,Francesca,B-NAME_STUDENT,O,B-,1
2640,22147,1408,Melanie,B-NAME_STUDENT,O,B-,1
2641,22181,738,Portman,B-NAME_STUDENT,O,B-,1
2642,22181,736,Natalie,B-NAME_STUDENT,O,B-,1
